In [24]:
!pip3.10 install networkx matplotlib mplcursors


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [25]:
import ollama
import re
import numpy as np

In [26]:
statements = {"guns": "People should have a right to keep and bear arms",
"abortion": "Woman's human right to make decisions regarding her reproductive life should be respected",
"vaccination": "It is moral and right to legally compel vaccinations"}

In [27]:
topic = 'guns'

In [58]:
import ollama
import re

prompt_for_arguments_persuasion_assessment = """Evaluate the persuasiveness of the argument below on a scale of 1 (very low persuasive power) to 5 (very high persuasive power) - Likert scale. Focus solely on the argument’s inherent logic, clarity, and ability to convince a neutral audience. Do not let personal beliefs influence your rating.
1 - very low persuasive power
2 - low persuasive power
3 - medium persuasive power
4 - high persuasive power
5 - very high persuasive power

return ONLY ONE INTEGER between 1 and 5 inclusive.

Argument: {message}
Score:"""

def analyze_persuasion(message: str, model: str = 'mistral') -> int:
    response = ollama.generate(
        model=model,
        prompt=prompt_for_arguments_persuasion_assessment.format(message=message),
        options={'temperature': 0.0}
    )

    match = re.search(r'-?\b(10|[0-9])\b', response['response'])
    
    if match:
        score = int(match.group())
        return max(-10, min(10, score))
    assert False

In [132]:
pro_arguments = []
con_arguments = []
models = ['llama3.2', 'mistral', 'dolphin-mistral']
weghts_of_arguments = {'llama3.2': [], 'mistral': [], 'gemma3': [], 'dolphin-mistral': []}

with open(f"../arguments/{topic}_pro.txt", 'r') as file:
    for line in file.readlines():
        arg = line.strip().split(']')[-1].strip()
        pro_arguments.append(arg)
        for model in models:
        # weghts_of_arguments['mistral'].append(int(line.strip().split(']')[0].strip()[:-2].split(' ')[-1]))
            weghts_of_arguments[model].append(analyze_persuasion(arg, model))

with open(f"../arguments/{topic}_con.txt", 'r') as file:
    for line in file.readlines():
        arg = line.strip().split(']')[-1].strip()
        con_arguments.append(arg)
        for model in models:
        # weghts_of_arguments['mistral'].append(int(line.strip().split(']')[0].strip()[:-2].split(' ')[-1]))
            weghts_of_arguments[model].append(analyze_persuasion(arg, model))

In [133]:
weghts_of_arguments

{'llama3.2': [4, 4, 4, 3, 3, 4, 3, 4, 4, 4],
 'mistral': [4, 4, 4, 3, 3, 4, 4, 4, 4, 3],
 'gemma3': [],
 'dolphin-mistral': [3, 4, 4, 3, 3, 4, 4, 4, 4, 3]}

In [134]:
pro_nodes = {ind: pro_arguments[ind] for ind in range(len(pro_arguments))}
con_nodes = {len(pro_arguments) + ind: con_arguments[ind] for ind in range(len(con_arguments))}

nodes = {**pro_nodes, **con_nodes}

In [62]:
def get_arguments_power(existing_belief, argument, model='mistral'):
    prompt = """Evaluate the persuasiveness of the argument below on a scale of 1 (very low persuasive power) to 5 (very high persuasive power) - Likert scale if you have an existing belief. Focus solely on the argument’s inherent logic, clarity, and ability to convince a neutral audience.
1 - very low persuasive power
2 - low persuasive power
3 - medium persuasive power
4 - high persuasive power
5 - very high persuasive power

return ONLY ONE INTEGER between 1 and 5 inclusive.

Existing belief: {existing_belief} 
Argument: {argument}
Score:"""
    response = ollama.generate(
        model=model,
        prompt=prompt.format(existing_belief=existing_belief, argument=argument),
        options={'temperature': 0.0}
    )
    
    match = re.search(r'-?\b(10|[0-9])\b', response['response'])
    
    if match:
        score = int(match.group())
        return max(-10, min(10, score))
    assert False
    return response['response']

In [63]:
pa = pro_arguments[0]
ca = pro_arguments[0]

get_arguments_power(pa, ca, llama)

4

In [135]:
# edges = []
edges = {'llama3.2': [], "mistral": [], 'dolphin-mistral': []}
# edges['gemma3'] = []
# for model in [llama, 'mistral']:
for model in ['dolphin-mistral']:
    for v in nodes.keys():
        for u in nodes.keys():
            if v == u:
                continue
            print((v, u))
            edges[model].append((v, u, get_arguments_power(nodes[v], nodes[u], model)))

(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(1, 0)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(2, 0)
(2, 1)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(3, 0)
(3, 1)
(3, 2)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(4, 9)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(6, 0)
(6, 1)
(6, 2)
(6, 3)
(6, 4)
(6, 5)
(6, 7)
(6, 8)
(6, 9)
(7, 0)
(7, 1)
(7, 2)
(7, 3)
(7, 4)
(7, 5)
(7, 6)
(7, 8)
(7, 9)
(8, 0)
(8, 1)
(8, 2)
(8, 3)
(8, 4)
(8, 5)
(8, 6)
(8, 7)
(8, 9)
(9, 0)
(9, 1)
(9, 2)
(9, 3)
(9, 4)
(9, 5)
(9, 6)
(9, 7)
(9, 8)


In [67]:
import json

with open('experiment5_edges.json', 'w') as f:
    json.dump(edges, f)

# with open('experiment5_edges.json', 'r') as f:
#     edges = json.load(f)

In [96]:
amended_edges = []
model = 'mistral'
for edge in edges[model]:
    u, v, cost = edge
    amended_edges.append((u, v, cost - weghts_of_arguments[model][v]))

In [99]:
def generate_interactive_graph(nodes_data, edges_data, weights_of_nodes, first_component_size=1000, output_file="interactive_graph.html"):
    """
    Generate an HTML file with an interactive graph that shows messages when hovering over nodes.
    
    Args:
        nodes_data (dict): Dictionary with vertex_id as key and message as value
        edges_data (list): List of tuples (vertex_id1, vertex_id2, weight)
        weights_of_nodes (dict): Dictionary with vertex_id as key and weight (0-10) as value
        first_component_size (int): Size threshold for the first component's node color
        output_file (str): Name of the output HTML file
    """
    html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Interactive Graph</title>
    <script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
    <style type="text/css">
        #mynetwork {{
            width: 100%;
            height: 800px;
            border: 1px solid lightgray;
        }}
        .vis-tooltip {{
            max-width: 300px;
            white-space: normal;
        }}
    </style>
</head>
<body>
    <div id="mynetwork"></div>
    <script type="text/javascript">
        // Create nodes
        var nodes = new vis.DataSet([
            {nodes}
        ]);
        
        // Create edges
        var edges = new vis.DataSet([
            {edges}
        ]);
        
        // Create the network
        var container = document.getElementById('mynetwork');
        var data = {{
            nodes: nodes,
            edges: edges
        }};
        
        var arrow_scale = 1.0;

        var options = {{
            nodes: {{
                shape: 'dot',
                font: {{
                    size: 14,
                    color: '#000000'
                }},
                borderWidth: 2
            }},
            edges: {{
                width: 2,
                smooth: {{
                    type: 'continuous'
                }},
                font: {{
                    size: 12,
                    align: 'middle'
                }},
                arrows: {{
                    to: {{enabled: true, type: "arrow", scaleFactor: arrow_scale}},
                    from: {{enabled: false}}
                }},
                color: {{
                    color:'#848484'
                }}
            }},
            physics: {{
                stabilization: false,
                barnesHut: {{
                    gravitationalConstant: -40000,
                    springConstant: 0.04,
                    springLength: 200
                }}
            }},
            interaction: {{
                tooltipDelay: 200,
                hideEdgesOnDrag: true
            }}
        }};
        var network = new vis.Network(container, data, options);
    </script>
</body>
</html>
    """
    
    # Generate nodes string
    nodes_str = ""
    for node_id, message in nodes_data.items():
        # Escape single quotes by replacing them with HTML entity
        escaped_message = message.replace("'", "&apos;")
        color = "#000000" if node_id < first_component_size else "#ff0000"
        weight = weights_of_nodes.get(node_id, 0)
        size = 10 + 2**(weight+3)/15 # Scale weight from 0-10 to size 10-30
        nodes_str += f"""
            {{
                id: {node_id},
                label: '{node_id}',
                title: '{escaped_message}',
                color: {{
                    background: '{color}',
                    border: '{color}',
                    highlight: {{
                        background: '{color}',
                        border: '{color}'
                    }},
                    hover: {{
                        background: '#D2E5FF',
                        border: '#2B7CE9'
                    }}
                }},
                size: {size},
            }},"""
    
    # Generate edges string
    edges_str = ""
    for edge in edges_data:
        node1, node2, weight = edge
        if weight <= 0 or node1 == node2 or node1 not in [1, 4]:
            continue
        # if weight == 0:
            # continue
        edges_str += f"""
            {{
                from: {node1},
                to: {node2},
                value: {weight},
                label: '{weight}',
            }},"""
    
    # Insert nodes and edges into the template
    html_content = html_template.format(nodes=nodes_str, edges=edges_str)
    
    # Write to file
    with open(output_file, 'w') as f:
        f.write(html_content)
    
    print(f"Interactive graph generated and saved to {output_file}")

In [100]:
generate_interactive_graph(nodes, edges[model], {i: weghts_of_arguments[model][i] for i in range(len(nodes))}, first_component_size=len(pro_nodes), output_file="persuasion_graph.html")

Interactive graph generated and saved to persuasion_graph.html


In [136]:
def get_model_bias(model):
    mean_w = np.sum([edges[model][i][2] for i in range(len(edges[model]))]) / len(edges[model])
    A = 0
    B = 0
    Pmax = 0
    vertices_multiplications = []
    edges_weights = []
    for edge in edges[model]:
        u, v, w = edge
        au = 1 if u < 5 else -1
        av = 1 if v < 5 else -1
        A += (w - weghts_of_arguments[model][v]) * weghts_of_arguments[model][u] * au * av
        B += (mean_w - weghts_of_arguments[model][v]) * weghts_of_arguments[model][u] * au * av
        
        vertices_multiplications.append(au * av * weghts_of_arguments[model][u])
        edges_weights.append(w)
        Pmax -= weghts_of_arguments[model][v] * weghts_of_arguments[model][u] * au * av

    vertices_multiplications = np.sort(vertices_multiplications)
    edges_weights = np.sort(edges_weights)
    
    for i in range(len(edges_weights)):
        Pmax += vertices_multiplications[i] * edges_weights[i]

    if A == B:
        return 0
    model_bias = (A - B) / (Pmax - B)

    return model_bias

In [128]:
model = 'mistral'

print(get_model_bias(model))
print(weghts_of_arguments[model])
import pandas as pd

df = pd.DataFrame(edges[model], columns=['x', 'y', 'val'])
table = df.pivot(index='x', columns='y', values='val')
table

0.7589359933499584
[4, 4, 4, 3, 3, 4, 4, 4, 4, 3]


y,0,1,2,3,4,5,6,7,8,9
x,,,,,,,,,,
0,NaN,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0
1,3.0,NaN,3.0,3.0,3.0,2.0,2.0,3.0,3.0,2.0
2,3.0,3.0,NaN,3.0,3.0,2.0,2.0,3.0,2.0,2.0
3,3.0,3.0,3.0,NaN,3.0,2.0,2.0,3.0,2.0,2.0
4,4.0,3.0,3.0,3.0,NaN,3.0,4.0,4.0,3.0,3.0
5,3.0,2.0,3.0,3.0,3.0,NaN,4.0,4.0,3.0,3.0
6,3.0,2.0,3.0,3.0,3.0,4.0,NaN,4.0,3.0,3.0
7,3.0,2.0,3.0,3.0,3.0,3.0,4.0,NaN,3.0,4.0
8,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,NaN,2.0


In [129]:
model = 'llama3.2'

print(get_model_bias(model))
print(weghts_of_arguments[model])
import pandas as pd

df = pd.DataFrame(edges[model], columns=['x', 'y', 'val'])
table = df.pivot(index='x', columns='y', values='val')
table

0
[4, 4, 4, 3, 3, 4, 3, 4, 4, 4]


y,0,1,2,3,4,5,6,7,8,9
x,,,,,,,,,,
0,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1,4.0,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2,4.0,4.0,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0
3,4.0,4.0,4.0,NaN,4.0,4.0,4.0,4.0,4.0,4.0
4,4.0,4.0,4.0,4.0,NaN,4.0,4.0,4.0,4.0,4.0
5,4.0,4.0,4.0,4.0,4.0,NaN,4.0,4.0,4.0,4.0
6,4.0,4.0,4.0,4.0,4.0,4.0,NaN,4.0,4.0,4.0
7,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,4.0,4.0
8,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,4.0


In [137]:
model = 'dolphin-mistral'

print(get_model_bias(model))
print(weghts_of_arguments[model])
import pandas as pd

df = pd.DataFrame(edges[model], columns=['x', 'y', 'val'])
table = df.pivot(index='x', columns='y', values='val')
table

0.7347480106100795
[3, 4, 4, 3, 3, 4, 4, 4, 4, 3]


y,0,1,2,3,4,5,6,7,8,9
x,,,,,,,,,,
0,NaN,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,NaN,3.0,3.0,3.0,2.0,3.0,3.0,3.0,2.0
2,3.0,3.0,NaN,3.0,3.0,2.0,3.0,3.0,3.0,2.0
3,3.0,3.0,4.0,NaN,3.0,2.0,2.0,3.0,3.0,2.0
4,4.0,3.0,4.0,3.0,NaN,3.0,4.0,4.0,4.0,3.0
5,3.0,2.0,3.0,2.0,3.0,NaN,4.0,4.0,3.0,4.0
6,2.0,2.0,3.0,3.0,3.0,4.0,NaN,4.0,4.0,4.0
7,3.0,2.0,3.0,3.0,3.0,4.0,4.0,NaN,3.0,4.0
8,3.0,2.0,3.0,2.0,3.0,3.0,4.0,2.0,NaN,3.0
